<a href="https://colab.research.google.com/github/kai-hun/project/blob/main/%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #시각화 라이브러리


#구글 드라이브 연결
from google.colab import drive
drive.mount("/content/gdrive")

#warning 메시지 ignore
import warnings
warnings.filterwarnings(action='ignore')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#데이터 호출
train_data = pd.read_table('', encoding='utf-8')
test_data = pd.read_table('', encoding='utf-8')

NameError: name 'pd' is not defined

In [ ]:
#결측치 제거
train_data.isnull().sum()
train_data = train_data.dropna()
test_data = test_data.dropna()

#중복 확인
train_data.info()
train_data['document'].nunique()

#중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
# 두 칸 이상의 빈칸 치환
train_data["document"]  = train_data["document"].str.replace("\s+", " ", regex=True)
test_data["document"]  = test_data["document"].str.replace("\s+", " ", regex=True)

#언어 설정
train_data['preprocessed'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['preprocessed'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
#토큰화
!pip install konlpy
from konlpy.tag import Okt

#토큰화 예시
okt = Okt() #konlpy에서 제공하는 형태소 분석기
okt.morphs('진짜 뭐냐 수업 너무 재밌네 어떡해', stem = True) #단어의 어간 추출하도록 함

#불용어 제거
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 37.0 MB/s eta 0:00:00


['진짜', '뭐', '냐', '수업', '너무', '재밌다', '어떡하다']

In [ ]:
import tqdm.notebook as tn

#트레인/테스트 데이터 정리
X_train = []
for sentence in tn.tqdm(train_data['preprocessed']):
    morphs = okt.pos(sentence, stem=True)  # 형태소 분석 (모든 품사 정보를 포함)
    selected_words = [morph for morph, pos in morphs if pos in ['Noun', 'Adjective', 'Verb']]  # 명사, 형용사, 동사만 선택
    temp = ' '.join(selected_words)
    X_train.append(temp)

train_data["tokenized_stem"] = pd.DataFrame(X_train) #기존 df에 삽입

X_test = []
for sentence in tn.tqdm(test_data['preprocessed']):
    morphs = okt.pos(sentence, stem=True)  # 형태소 분석 (모든 품사 정보를 포함)
    selected_words = [morph for morph, pos in morphs if pos in ['Noun', 'Adjective', 'Verb']]  # 명사, 형용사, 동사만 선택
    temp = ' '.join(selected_words)
    X_test.append(temp)

y_train=np.array(train_data['label'])
y_test=np.array(test_data['label'])

In [ ]:
#벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer="word", ngram_range =(1,3)) #단어 단위 변환
X_train_vector = tfidf.fit_transform(X_train)
X_test_vector = tfidf.transform(X_test)

In [ ]:
#로지스틱 회귀
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(class_weight ='balanced') #객체에 모델 할당
model.fit(X_train_vector, y_train) #모델 학습

y_pred = model.predict(X_test_vector) #학습된 모델을 test에 적용해서 예측된 라벨 얻기

# 정확도 계산
from sklearn.metrics import accuracy_score, f1_score, log_loss

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

In [ ]:
import tqdm.notebook as tn

#분류 함수 정의
def preprocess_for_prediction(sentence):
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', sentence)
    morphs = okt.pos(sentence, stem=True)
    selected_words = [morph for morph, pos in morphs if pos in ['Noun', 'Adjective', 'Verb']]
    preprocessed_sentence = ' '.join(selected_words)
    return preprocessed_sentence

def sentiment_predict(new_sentence, tfidf_model, model):
    preprocessed_sentence = preprocess_for_prediction(new_sentence)
    tfidf_vector = tfidf_model.transform([preprocessed_sentence])

    score = float(model.predict(tfidf_vector))

    if score == 1:
        print("긍정 리뷰입니다.\n")
    else:
        print("부정 리뷰입니다.\n")

# 예측할 문장 입력
new_sentence = input('분석할 문장을 입력 : ')
sentiment_predict(new_sentence, tfidf, model)